In [106]:
import re
import json
import logging
import pandas as pd
import selenium
import time
import sys
import linecache
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import pymongo
import datetime
import random
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from time import sleep 
from pymongo import MongoClient 
from selenium.webdriver.common.by import By

## KLIK INDOMARET SCRAPE

In [4]:
# Print exception
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print ('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [103]:
# Scraping the page
def scrape_detail_page(url,category,collection):
    time.sleep(10)
    # Connect to mongodb
    try: 
        conn = MongoClient() 
        print("Connected successfully!!!") 
    except:   
        print("Could not connect to MongoDB") 
    # database
    db = conn.klik_indomaret
    # Created or Switched to collection names: pakaian_wanita 
    collection = db[collection]

    # Load WebDriver and navigate to the page url.
    # This will open a browser window.
    driver = webdriver.Firefox()
    driver.get(url)
    result = pd.DataFrame()
    while(True):
        path = driver.find_elements_by_css_selector('.item')
        try:
            for x in range(4,len(path)):
                try:
                    discount_price = re.sub("\.","",re.sub("Rp","",path[x].find_element_by_css_selector('.price-value').text))
                    origin_price = re.sub("\.","",re.sub("Rp","",re.sub(" [0-9].%","",path[x].find_element_by_css_selector('.price > .disc-price').text)))

                except:
                    origin_price = re.sub("\.","",re.sub("Rp","",path[x].find_element_by_css_selector('.price-value').text))
                    discount_price = origin_price
                name = path[x].find_element_by_css_selector('.title').text
                image = path[x].find_element_by_css_selector('.wrp-media > img').get_attribute('src')
                sku = path[x].find_element_by_css_selector('.sendby').get_attribute('selecteds')

                time_check = 0
                while(image is None and time_check < 60):
                    time_check = time_check+1
                    path[x].find_element_by_css_selector('.title').location_once_scrolled_into_view
                    time.sleep(1)
                    image = path[x].find_element_by_css_selector('.wrp-media > img').get_attribute('src')
                    print("wait_image" + " " + name)

                # Return result
                date = datetime.datetime.now().strftime("%Y-%m-%d")
                new_result = pd.DataFrame([{'sku':sku,'name':name, 'origin_price':origin_price, 'discount_price':discount_price, 'image':image,'category':category,'scraping_date':date}])
                result = result.append(new_result)

                # save to mongodb
                collection.insert_many(new_result.to_dict('records'))
                
            # next page
            try:
                check_prev = driver.find_element_by_css_selector('.pagination > div > a > .prev')
                check_next = driver.find_element_by_css_selector('.pagination > div > a > .next')
                next = driver.find_elements_by_css_selector('.pagination > div > a')[1]
            except:
                check_next = driver.find_element_by_css_selector('.pagination > div > a > .next')
                next = driver.find_elements_by_css_selector('.pagination > div > a')[0]
            next.click()
        except:
            PrintException()
            break
    driver.close()
    return result


In [105]:
# MAIN
klikindomaret_category = pd.read_csv('..\\Data\\klikindomaret_category.csv',sep=",")
result = pd.DataFrame()
for x in range(0,len(klikindomaret_category)):
    category = klikindomaret_category.iloc[x]
    print(str(x) + "/" + str(len(klikindomaret_category)) + " " + category['collection'] + " - " + category['category'])
    
    url = category['link']
    collection = category['collection']
    
    result = scrape_detail_page(url,category['category'],collection)
    print(result.head())

0/58 produk_segar - nugget
Connected successfully!!!
wait_image Belfoods Royal Chicken Stick 500G
EXCEPTION IN (<ipython-input-103-c734ee278945>, LINE 57 "check_next = driver.find_element_by_css_selector('.pagination > div > a > .next')"): Message: Unable to locate element: .pagination > div > a > .next

  category discount_price                                              image  \
0   nugget          35900  https://assets.klikindomaret.com/products/1001...   
0   nugget          35900  https://assets.klikindomaret.com/products/1002...   
0   nugget          33000  https://assets.klikindomaret.com/products/2005...   
0   nugget          48500  https://assets.klikindomaret.com/products/2002...   
0   nugget          56000  https://assets.klikindomaret.com/products/1002...   

                                          name origin_price scraping_date  \
0       Belfoods Favorite Chicken Nuggets 500G        42500    2019-09-17   
0         Belfoods Favorite Chicken Stick 500G        42500

Connected successfully!!!
wait_image Telur Ayam Negeri (Harga Per-Butir)
EXCEPTION IN (<ipython-input-103-c734ee278945>, LINE 57 "check_next = driver.find_element_by_css_selector('.pagination > div > a > .next')"): Message: Unable to locate element: .pagination > div > a > .next

  category discount_price                                              image  \
0    telur           1550  https://assets.klikindomaret.com/products/2005...   
0    telur           2650  https://assets.klikindomaret.com/products/1000...   

                                   name origin_price scraping_date  \
0   Telur Ayam Negeri (Harga Per-Butir)         1550    2019-09-17   
0  Telur Ayam Omega 3 (Harga Per-Butir)         2650    2019-09-17   

                                    sku  
0  76d12fc5-b2dc-421a-85fb-ad70ec611652  
0  084b89fb-9f8d-41aa-9dd5-0c50acdba308  
7/58 produk_segar - es-krim-stick--cup
Connected successfully!!!
wait_image Campina Ice Cream Summer Barz Frozen Berry 55Ml
wait_image Wall's

Connected successfully!!!
EXCEPTION IN (<ipython-input-103-c734ee278945>, LINE 57 "check_next = driver.find_element_by_css_selector('.pagination > div > a > .next')"): Message: Unable to locate element: .pagination > div > a > .next

  category discount_price                                              image  \
0    donut          79000  https://assets.klikindomaret.com/products/2008...   
0    donut          79000  https://assets.klikindomaret.com/products/2008...   
0    donut         145000  https://assets.klikindomaret.com/products/2008...   
0    donut         145000  https://assets.klikindomaret.com/products/2008...   
0    donut         145000  https://assets.klikindomaret.com/products/2008...   

                   name origin_price scraping_date  \
0  Mister Donut Combo 2        79000    2019-09-17   
0  Mister Donut Combo 1        79000    2019-09-17   
0  Mister Donut Combo 4       145000    2019-09-17   
0  Mister Donut Combo 5       145000    2019-09-17   
0  Mister Donut

  category discount_price                                              image  \
0   kornet          20800  https://assets.klikindomaret.com/products/1003...   
0   kornet           5600  https://assets.klikindomaret.com/products/2006...   
0   kornet           5600  https://assets.klikindomaret.com/products/2006...   
0   kornet          20700  https://assets.klikindomaret.com/products/1000...   
0   kornet          29200  https://assets.klikindomaret.com/products/1003...   

                                        name origin_price scraping_date  \
0                       Cip Corned Beef 198G        20800    2019-09-17   
0  Kimbo Sosis Lauk Mi Goreng Ayam Bakar 60G         5600    2019-09-17   
0     Kimbo Sosis Lauk Mi Kuah Baso Sapi 60G         5600    2019-09-17   
0                    Pronas Corned Beef 198G        20700    2019-09-17   
0                    Pronas Corned Beef 340G        29200    2019-09-17   

                                    sku  
0  84b90b1a-2f14-4482-ae08

Connected successfully!!!
wait_image Delfi Black Thunder Choco 21G
wait_image Fitbar Chocolate Fitbar 25G
wait_image Silver Queen Chocolate Midi Merah/Biru 33G
EXCEPTION IN (<ipython-input-103-c734ee278945>, LINE 57 "check_next = driver.find_element_by_css_selector('.pagination > div > a > .next')"): Message: Unable to locate element: .pagination > div > a > .next

  category discount_price                                              image  \
0  cokelat          23500  https://assets.klikindomaret.com/products/2005...   
0  cokelat           9100  https://assets.klikindomaret.com/products/2009...   
0  cokelat           9000  https://assets.klikindomaret.com/products/2009...   
0  cokelat          23600  https://assets.klikindomaret.com/products/2009...   
0  cokelat           6000  https://assets.klikindomaret.com/products/2009...   

                                                name origin_price  \
0               L'agie Milk Chocolate Cashew Nut 90G        23500   
0    Delfi Gr

Connected successfully!!!
EXCEPTION IN (<ipython-input-103-c734ee278945>, LINE 57 "check_next = driver.find_element_by_css_selector('.pagination > div > a > .next')"): Message: Unable to locate element: .pagination > div > a > .next

     category discount_price  \
0  sup-instan           8400   
0  sup-instan           8400   
0  sup-instan           8700   
0  sup-instan           8700   
0  sup-instan           3100   

                                               image  \
0  https://assets.klikindomaret.com/products/1003...   
0  https://assets.klikindomaret.com/products/1003...   
0  https://assets.klikindomaret.com/products/2004...   
0  https://assets.klikindomaret.com/products/2004...   
0  https://assets.klikindomaret.com/products/2001...   

                                    name origin_price scraping_date  \
0                Royco Sup Krim Ayam 58G         8400    2019-09-17   
0          Royco Sup Krim Jagung 50G Pck         8400    2019-09-17   
0  Kimbo Kitchen Bubur 

Connected successfully!!!
wait_image Cheetos Snack Jagung Bakar 75G
wait_image Kusuka Keripik Singkong Saos Balado 60G
wait_image Potabee Snack Potato Chips Salted Egg 68G
EXCEPTION IN (<ipython-input-103-c734ee278945>, LINE 57 "check_next = driver.find_element_by_css_selector('.pagination > div > a > .next')"): Message: Unable to locate element: .pagination > div > a > .next

  category discount_price                                              image  \
0  keripik          10500  https://assets.klikindomaret.com/promos/201909...   
0  keripik          10500  https://assets.klikindomaret.com/promos/201909...   
0  keripik           5300  https://assets.klikindomaret.com/promos/201909...   
0  keripik           6000  https://assets.klikindomaret.com/promos/201909...   
0  keripik           6000  https://assets.klikindomaret.com/promos/201909...   

                                           name origin_price scraping_date  \
0     Chitato Snack Potato Chips Ayam Bumbu 68G        10500 

Connected successfully!!!
wait_image Entrasol Active Susu Bubuk Chocolate 160g
wait_image Appeton Susu Bubuk 60+ Vanilla 900g
wait_image Milo Healty Drink 3 In 1 Actigen-E 4X35g
EXCEPTION IN (<ipython-input-103-c734ee278945>, LINE 57 "check_next = driver.find_element_by_css_selector('.pagination > div > a > .next')"): Message: Unable to locate element: .pagination > div > a > .next

               category discount_price  \
0  susu--kesehatan-diet          24100   
0  susu--kesehatan-diet          58200   
0  susu--kesehatan-diet         125400   
0  susu--kesehatan-diet          60350   
0  susu--kesehatan-diet          30700   

                                               image  \
0  https://assets.klikindomaret.com/products/2007...   
0  https://assets.klikindomaret.com/products/2001...   
0  https://assets.klikindomaret.com/products/2000...   
0  https://assets.klikindomaret.com/products/2001...   
0  https://assets.klikindomaret.com/products/2002...   

                        

Connected successfully!!!
EXCEPTION IN (<ipython-input-103-c734ee278945>, LINE 57 "check_next = driver.find_element_by_css_selector('.pagination > div > a > .next')"): Message: Unable to locate element: .pagination > div > a > .next

              category discount_price  \
0  coklat-bubuk-kremer          57900   
0  coklat-bubuk-kremer           7900   
0  coklat-bubuk-kremer           6600   
0  coklat-bubuk-kremer           8500   
0  coklat-bubuk-kremer           8700   

                                               image  \
0  https://assets.klikindomaret.com/products/2002...   
0  https://assets.klikindomaret.com/products/2008...   
0  https://assets.klikindomaret.com/products/2009...   
0  https://assets.klikindomaret.com/products/2007...   
0  https://assets.klikindomaret.com/products/2008...   

                                            name origin_price scraping_date  \
0        Cadbury Hot Chocolate Drink 3 In 1 450G        67500    2019-09-17   
0                Beng-Be

Connected successfully!!!
wait_image Indomaret Teh Hijau Celup Melati 25X2g
EXCEPTION IN (<ipython-input-103-c734ee278945>, LINE 57 "check_next = driver.find_element_by_css_selector('.pagination > div > a > .next')"): Message: Unable to locate element: .pagination > div > a > .next

    category discount_price  \
0  teh-celup          35900   
0  teh-celup          35900   
0  teh-celup          35900   
0  teh-celup          35900   
0  teh-celup           7800   

                                               image  \
0  https://assets.klikindomaret.com/products/2008...   
0  https://assets.klikindomaret.com/products/2008...   
0  https://assets.klikindomaret.com/products/2003...   
0  https://assets.klikindomaret.com/products/2008...   
0  https://assets.klikindomaret.com/products/2007...   

                                        name origin_price scraping_date  \
0        Tasseo Teh Celup Java Jasmine 20X2g        50900    2019-09-17   
0   Tasseo Teh Celup English Breakfast 20X

## After Scraping

In [107]:
def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]


def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db_out = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db_out[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if '_id' in df.columns:
        del df['_id']
    
    if db == 'visenze_tagging':
        del df['created_date'], df['metadata'], df['box']
    return df

In [108]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

dbname = 'klik_indomaret'
mydb = myclient[dbname]
result = pd.DataFrame()

#list the collections
for coll in mydb.collection_names():
    data = read_mongo(dbname,coll)
    data['collection'] = coll
    data = data
    result = result.append(data)

result = result.drop_duplicates(['name','image'])
filename = "INDOMARET_"+str(datetime.datetime.now().strftime("%Y%m%d"))+'.csv'
result.to_csv('..\\Result\\'+filename,index=False)